In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_DIR = "./drive/MyDrive/NLP/"

In [2]:
# !pip install tqdm boto3 requests regex sentencepiece sacremoses
# !pip install transformers
# !pip install segmentation-models-pytorch

In [3]:
!pip install segmentation-models-pytorch

  Using cached segmentation_models_pytorch-0.2.1-py3-none-any.whl (88 kB)
  Using cached pretrainedmodels-0.7.4.tar.gz (58 kB)
  Using cached efficientnet_pytorch-0.6.3.tar.gz (16 kB)
  Using cached timm-0.4.12-py3-none-any.whl (376 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=97b9529212c67261699ece3cca5caa7476bbc86b3d712ec041defd8cdb837306
  Stored in directory: c:\users\larry\appdata\local\pip\cache\wheels\70\f8\49\20f330df3f946fed839df657dd2156c929d6d7b5f774d9650e
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=390b1450f7b6ac22d23a6695f977f668534252aac3339f85495eba71a24415a9
  Stored in directory: c:\users\larry\appdata\local\pip\cache\wheels\d1\3b\4e\2f3015f1ab76f34be28e04c4bcee27e8cabfa70d2eadf8bc3b
Successfully built efficientnet-pytorch pretrainedmodels


In [4]:
# load the data
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np

DATA_DIR = ""
fname = "cmv.tar.bz2"
# url = "https://chenhaot.com/data/cmv/" + fname

# download if not exists
# if not os.path.isfile(fname):
#     f = BytesIO()
#     with request.urlopen(url) as resp, open(fname, 'wb') as f_disk:
#         data = resp.read()
#         f_disk.write(data)  # save to disk too
#         f.write(data)
#         f.seek(0)
# else:
f = open(DATA_DIR+fname, 'rb')


tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

all_train_fname = "all/train_period_data.jsonlist.bz2"
all_test_fname = "all/heldout_period_data.jsonlist.bz2"

pair_train_fname = "pair_task/train_pair_data.jsonlist.bz2"
pair_test_fname = "pair_task/train_pair_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)
# all_train_bzlist = tar.extractfile(all_train_fname)
# pair_train_bzlist = tar.extractfile(pair_train_fname)

# Deserialize the JSON list
original_posts_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]
# all_train = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(all_train_bzlist)
# ]
# pair_train = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(pair_train_bzlist)
# ]
test_bzlist = tar.extractfile(test_fname)
# all_test_bzlist = tar.extractfile(all_test_fname)
# pair_test_bzlist = tar.extractfile(pair_test_fname)

original_posts_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(test_bzlist)
]
# all_test = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(all_test_bzlist)
# ]
# pair_test = [
#     json.loads(line.decode('utf-8'))
#     for line in BZ2File(pair_test_bzlist)
# ]
f.close()

In [5]:
malleable_op = [post["name"] for post in original_posts_train if post["delta_label"]]
print(f'{len(original_posts_train)}, {len(malleable_op)}')

10743, 3191


In [6]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score


In [7]:
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
            ]
    return "\n".join(lines)

In [8]:
op_text_train = [
    cleanup(original_post['selftext'])
    # original_post['selftext']  # try this at home
    for original_post
    in original_posts_train
]
op_text_test = [
    cleanup(original_post['selftext'])
    # original_post['selftext']  # try this at home
    for original_post
    in original_posts_test
]

In [9]:
vect = TfidfVectorizer(use_idf=False, norm='l1')
X_train = vect.fit_transform(op_text_train)

In [10]:
lr = LogisticRegressionCV(Cs=10, class_weight='balanced',
                          scoring='roc_auc', solver='sag',
                          tol=0.001, max_iter=500,
                          random_state=0)

In [11]:
X_train.shape

(10743, 48296)

In [12]:
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])

lr.fit(X_train, is_malleable_train)

LogisticRegressionCV(class_weight='balanced', max_iter=500, random_state=0,
                     scoring='roc_auc', solver='sag', tol=0.001)

In [13]:
X_test = vect.transform(op_text_test)
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
test_roc = roc_auc_score(is_malleable_test,lr.predict_proba(X_test)[:,1])
print("Test ROC AUC score: {:.3f}".format(test_roc))

Test ROC AUC score: 0.531


## Longformer Tokenization + UNET

In [14]:
import pandas as pd
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [15]:
config = LongformerConfig()

config

LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.17.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [16]:
# load model and tokenizer and define length of the text sequence
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)

In [17]:
example_text = op_text_train[0]
long_input = tokenizer(example_text,padding='max_length', max_length = 1024, 
                       truncation=True, return_tensors="pt")

print(long_input['input_ids'])
print(long_input['attention_mask'])

tensor([[  0, 100, 206,  ...,   1,   1,   1]])
tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [18]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [int(label) for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 1024, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [115]:
import pandas as pd
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])
df_train_total = pd.DataFrame(list(zip(op_text_train,is_malleable_train)),columns = ['text','label'])
df_train, df_val = np.split(df_train_total.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_train_total))])
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = pd.DataFrame(list(zip(op_text_test,is_malleable_test)),columns = ['text','label'])

In [118]:
print("tokenizing data")
train = Dataset(df_train)
val = Dataset(df_val)

tokenizing data


In [75]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=False)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

In [141]:
import segmentation_models_pytorch as smp

class UNET_Classifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(UNET_Classifier, self).__init__()

        self.unet = smp.Unet(
            encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=2,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=1,                      # model output channels (number of classes in your dataset)
        )
        self.pool = nn.MaxPool2d(2, 2)
        self.linear = nn.Linear(256,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
            
        input_2d = input_id.reshape((-1,1,32,32)).float()
        mask_2d = mask.reshape((-1,1,32,32)).float()
        final_input = torch.cat((input_2d,mask_2d),dim=1)
        out = self.unet(final_input)
        out = self.pool(out)
        out = self.relu(out)
        out= out.reshape((-1,256))
        out = self.linear(out)
        out = self.sigmoid(out).squeeze()
        return out

In [147]:
def run_test(train, val, lr=0.001, BATCH_SIZE=16, num_epochs = 50):
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = UNET_Classifier()
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr= lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()
#     print("start training")

    best_val_loss = 100
    
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE)
    
    for epoch_num in range(num_epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].to(device)

                output = model(input_id, mask)
                batch_loss = criterion(output.squeeze(), train_label.float())
                total_loss_train += batch_loss.item()

                acc = (torch.round(output) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            scheduler.step()

    total_acc_val = 0
    total_loss_val = 0

    with torch.no_grad():

        for val_input, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output.squeeze(), val_label.float())
            total_loss_val += batch_loss.item()

            acc = (torch.round(output) == val_label).sum().item()
            total_acc_val += acc

#             print(
#                 f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
#                 | Train Accuracy: {total_acc_train / len(train): .3f} \
#                 | Val Loss: {total_loss_val / len(val): .3f} \
#                 | Val Accuracy: {total_acc_val / len(val): .3f}')
#             if (total_loss_val/len(val))<best_val_loss:
#                 best_val_loss = total_loss_val/len(val)
#                 print("New best val loss, model saved!")
#                 torch.save(model,'best_model_unet')
    return total_acc_val / len(val), model

In [148]:
df_train_total

,text,label
0,I think the world is automating fast enough th...,False
1,"First off, I do not believe that Iran *should*...",False
2,Please leave the footnote below the following ...,False
3,It seems to me that it is entirely justified t...,False
4,"The, ""Under God"" line is actually a relatively...",False
...,...,...
10738,Muhammad did not prohibit paintings of him bec...,False
10739,It seems logical to me that school hours shoul...,True
10740,I think that dynastic wealth i.e. passing down...,False
10741,cars are a bad idea. just like oil is a bad id...,False


In [149]:
df_cv = df_train_total[:10740]
df_cv

,text,label
0,I think the world is automating fast enough th...,False
1,"First off, I do not believe that Iran *should*...",False
2,Please leave the footnote below the following ...,False
3,It seems to me that it is entirely justified t...,False
4,"The, ""Under God"" line is actually a relatively...",False
...,...,...
10735,The rate at which income is taxed (at least in...,True
10736,"Charlie Hebdo is staffed exclusively by white,...",False
10737,Certain limitations of freedom of speech are c...,False
10738,Muhammad did not prohibit paintings of him bec...,False


In [158]:
val_size = int(len(df_cv)/5)
max_val_acc = 0
file_name = 'unet_results.txt'

def run_CV(df_train, lr, BATCH_SIZE, NUM_EPOCHS, max_val_acc):   
    final_acc_val = np.zeros(5)
    print("lr =", lr, ", batch size =", BATCH_SIZE,", num_epochs =", NUM_EPOCHS)
    print("lr =", lr, ", batch size =", BATCH_SIZE, ", num_epochs =", NUM_EPOCHS, file=open(file_name, "a"))
    for i in range(5):
        
        train_temp = pd.concat([df_train[0:i*val_size],df_train[(i+1)*val_size:]]).reset_index(drop=True)
        val_temp = df_train[i*val_size:(i+1)*val_size].reset_index(drop=True)
        train = Dataset(train_temp)
        val = Dataset(val_temp)
        val_acc, model = run_test(train, val, lr=lr, BATCH_SIZE=BATCH_SIZE, num_epochs = NUM_EPOCHS)
        final_acc_val[i] = val_acc
        
#         losses, losses_val, model  = run_test(X_train_temp,y_train_temp,X_val_temp,y_val_temp,TASK=TASK, lr=lr, 
#                                       BATCH_SIZE=BATCH_SIZE, num_epochs=NUM_EPOCHS)
#         final_loss_train[i] = losses[-1]
#         final_loss_val[i] = losses_val[-1]
        
    
    print("average validation acc:", final_acc_val.mean())
    print("average validation acc:", final_acc_val.mean(), file=open(file_name, "a"))
    
    if final_acc_val.mean() > max_val_acc:
        max_val_acc = final_acc_val.mean()
        print('best params:',  "lr =", lr, ", batch size =", BATCH_SIZE, ", num_epochs =", NUM_EPOCHS)
        print('best params:',  "lr =", lr, ", batch size =", BATCH_SIZE, ", num_epochs =", NUM_EPOCHS, file=open(file_name, "a"))
        torch.save(model,'best_model_unet')
        print('new best model saved!')
    return max_val_acc

In [160]:
lr_grid = [0.0001, 0.001, 0.005, 0.01]
BATCH_SIZE_grid = [8, 16, 32, 64]
num_eps_grid = [5,10,20,50]
# dropout_grid = [0.2, 0.4, 0.6]
max_val_acc = 0
# here we dont use scheduler, 
# training for 600 epochs
for lr in lr_grid:
    for BATCH_SIZE in BATCH_SIZE_grid:
        for epoch in num_eps_grid:
            max_val_acc = run_CV(df_cv,lr, BATCH_SIZE, NUM_EPOCHS = epoch, max_val_acc=max_val_acc)

lr = 0.0001 , batch size = 8 , num_epochs = 5
average validation acc: 0.7027001862197393
best params: lr = 0.0001 , batch size = 8 , num_epochs = 5
new best model saved!
lr = 0.0001 , batch size = 8 , num_epochs = 10
average validation acc: 0.6965549348230912
lr = 0.0001 , batch size = 8 , num_epochs = 20
average validation acc: 0.6182495344506518
lr = 0.0001 , batch size = 8 , num_epochs = 50
average validation acc: 0.6116387337057729
lr = 0.0001 , batch size = 16 , num_epochs = 5
average validation acc: 0.6971135940409683
lr = 0.0001 , batch size = 16 , num_epochs = 10
average validation acc: 0.639292364990689
lr = 0.0001 , batch size = 16 , num_epochs = 20
average validation acc: 0.5952513966480447
lr = 0.0001 , batch size = 16 , num_epochs = 50
average validation acc: 0.5844506517690875
lr = 0.0001 , batch size = 32 , num_epochs = 5
average validation acc: 0.6818435754189944
lr = 0.0001 , batch size = 32 , num_epochs = 10
average validation acc: 0.5938547486033519
lr = 0.0001 , bat

KeyboardInterrupt: 

In [162]:
from torch.optim import Adam
from tqdm import tqdm
EPOCHS = 50
model = UNET_Classifier()
LR = 1e-3 
train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=False)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= LR)

if use_cuda:

        model = model.cuda()
        criterion = criterion.cuda()
print("start training")

best_val_loss = 100
for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output.squeeze(), train_label.float())
            total_loss_train += batch_loss.item()
            
            acc = (torch.round(output) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output.squeeze(), val_label.float())
                total_loss_val += batch_loss.item()
                
                acc = (torch.round(output) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
            | Train Accuracy: {total_acc_train / len(train): .3f} \
            | Val Loss: {total_loss_val / len(val): .3f} \
            | Val Accuracy: {total_acc_val / len(val): .3f}')
        if (total_loss_val/len(val))<best_val_loss:
            best_val_loss = total_loss_val/len(val)
            print("New best val loss, model saved!")
            torch.save(model,'best_model_unet')
                  
  

start training


100%|██████████| 269/269 [00:07<00:00, 34.13it/s]


Epochs: 1 | Train Loss:  0.019             | Train Accuracy:  0.698             | Val Loss:  0.019             | Val Accuracy:  0.715
New best val loss, model saved!


100%|██████████| 269/269 [00:07<00:00, 34.97it/s]


Epochs: 2 | Train Loss:  0.019             | Train Accuracy:  0.700             | Val Loss:  0.019             | Val Accuracy:  0.715


...

Epochs: 48 | Train Loss:  0.002             | Train Accuracy:  0.978             | Val Loss:  0.093             | Val Accuracy:  0.625


100%|██████████| 269/269 [00:08<00:00, 33.09it/s]


Epochs: 49 | Train Loss:  0.002             | Train Accuracy:  0.973             | Val Loss:  0.107             | Val Accuracy:  0.649


100%|██████████| 269/269 [00:08<00:00, 33.02it/s]


Epochs: 50 | Train Loss:  0.002             | Train Accuracy:  0.978             | Val Loss:  0.104             | Val Accuracy:  0.642


In [163]:
test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

In [164]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    model = model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)

          acc = (torch.round(output) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

100%|██████████| 48/48 [00:00<00:00, 88.63it/s]

Test Accuracy:  0.545


In [165]:
best_model = torch.load('best_model_unet')


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    best_model = best_model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = best_model(input_id, mask)

          acc = (torch.round(output) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

100%|██████████| 48/48 [00:00<00:00, 97.23it/s] 

Test Accuracy:  0.560


## Previous try with BERT base

In [ ]:
import torch
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', './test/bert_saved_model/')  # E.g. tokenizer was saved using `save_pretrained('./test/saved_model/')`

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = op_text_train[0]
bert_input = tokenizer(example_text,padding='max_length', max_length = 5000, 
                       truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101,  146, 1341,  ...,    0,    0,    0]])
tensor([[0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [ ]:
lengths = np.array([len(text.split()) for text in op_text_train])
print(lengths.max())
print(lengths.argmax())
print(op_text_train[224]) # Longest OP text

5511
224
## Section I: Definitions of "Social Democracy" and "Socialism"

Social democracy "is defined as a policy regime involving a universal welfare state and collective bargaining schemes within the framework of a capitalist economy. It is often used in this manner to refer to the social models and economic policies prominent in Western and Northern Europe during the later half of the 20th century."

I really like the political economic structures of Scandinavian nations. The basic idea of a "social democracy" is that "the means of production are mainly under private ownership; that markets remain the dominant form of economic coordination; and that profit-seeking self-interested enterprises and the accumulation of capital remains the fundamental driving force behind economic activity. However, unlike a laissez-faire economy, the government weilds considerable indirect influence over the economy through fiscal and monetary policies designed to counteract economic downturns and capi

In [ ]:
print(np.sort(lengths)[-5000:])

[ 201  201  201 ... 2429 3595 5511]


In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] I think the world is automating fast enough that a utopia will arise where no one will have to work anymore. Within the next 2 decades or so, having a job won't mean much, and most people will be artists and scientists. My parents let me live with them, so I can just wait until the utopia happens. CMV. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [int(label) for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
import pandas as pd
df_train = pd.DataFrame(list(zip(op_text_train,is_malleable_train)),columns = ['text','label'])
df_train, df_val = np.split(df_train.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_train))])
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = pd.DataFrame(list(zip(op_text_test,is_malleable_test)),columns = ['text','label'])

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

In [ ]:
is_malleable_train = np.array([post["delta_label"] for post in original_posts_train])
is_malleable_test = np.array([post["delta_label"] for post in original_posts_test])

In [ ]:
print("tokenizing data")
train = Dataset(df_train)
val = Dataset(df_val)

tokenizing data


In [ ]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=False)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

In [ ]:
from torch.optim import Adam
from tqdm import tqdm
EPOCHS = 5
model = BertClassifier()
LR = 1e-6 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= LR)

if use_cuda:

        model = model.cuda()
        criterion = criterion.cuda()
print("start training")
for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output.squeeze(), train_label.float())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output.squeeze(), val_label.float())
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
            | Train Accuracy: {total_acc_train / len(train): .3f} \
            | Val Loss: {total_loss_val / len(val): .3f} \
            | Val Accuracy: {total_acc_val / len(val): .3f}')
                  
  

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.4.attention.out_lin.weight', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'distilbert.transformer.layer.5.attention.k_lin.weight', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.3.attention.q_lin.weight', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.5.output_layer_norm.bias', 'distilbert.transformer.layer.1.output_layer_norm.bias', 'distilbert.transformer.layer.2.sa_layer_norm.bias', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.tran

start training


  0%|          | 13/4297 [02:55<16:03:29, 13.49s/it]


KeyboardInterrupt: ignored

In [ ]:
test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

In [ ]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    model = model.cuda()

total_acc_test = 0
with torch.no_grad():

    for test_input, test_label in test_dataloader:

          test_label = test_label.to(device)
          mask = test_input['attention_mask'].to(device)
          input_id = test_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)

          acc = (output.argmax(dim=1) == test_label).sum().item()
          total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(test): .3f}')

In [ ]:
def top_features(coef, words, n=15):
    order = np.argsort(-coef)
    for i_hi, i_lo in zip(order[:n], order[:-n - 1:-1]):
        row_vals = coef[i_hi], words[i_hi], coef[i_lo], words[i_lo]
        print("[{:>4.1f}] {:<20}[{:>4.1f}] {:<20}".format(*row_vals))

In [ ]:
top_features(lr.coef_.flatten(), vect.get_feature_names_out())